# Activites List
- It is highly encouraged to work in a group of two students for the lab of this case study.
- <b>Important: for Activity 1, Activity 2 and  Activity 3 , please use the files [file1.csv](./Data/file1.csv), [file2.csv](./Data/file2.csv) and [file3.csv](./Data/file3.csv) from the [Data](./Data) folder.</b>

### Activity 1 (Monday)

- Aggregate data into one Data Frame using Pandas. Pay attention that files may have different names for the same column. therefore, make sure that you unify the columns names before concating them. 
- Standardizing header names
- Deleting and rearranging columns – delete the column customer as it is only a unique identifier for each row of data
- Working with data types – Check the data types of all the columns and fix the incorrect ones (for ex. customer lifetime value and number of open complaints ). Hint: remove the percentage from the customer lifetime value and truncate it to an integer value.
- clean the number of open complaints and extract the middle number which is changing between records. pay attention that the number of open complaints is a categorical feature.
- Filtering data and Correcting typos – Filter the data in state and gender column to standardize the texts in those columns
- Removing duplicates

In [1]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import boxcox
from itertools import compress

pd.options.display.max_rows = 100
## Install xlrd package to load Excel files
#!conda install openpyxl
#!conda install xlrd

In [2]:
def clean_columns():
    """
    clean column names;
    cast all as lowercase;
    replace "st" with "state";
    replace all spaces with "_"
    """
    f1 = pd.read_csv(r'C:\Users\p.kollhof\Documents\IRONHACK_GitHub\IH_RH_DA_FT_AUG_2022\Class_Materials\Case_Studies\Customer_Analysis_Case_Study\Data\file1.csv')
    f2 = pd.read_csv(r'C:\Users\p.kollhof\Documents\IRONHACK_GitHub\IH_RH_DA_FT_AUG_2022\Class_Materials\Case_Studies\Customer_Analysis_Case_Study\Data\file2.csv')
    f3 = pd.read_csv(r'C:\Users\p.kollhof\Documents\IRONHACK_GitHub\IH_RH_DA_FT_AUG_2022\Class_Materials\Case_Studies\Customer_Analysis_Case_Study\Data\file3.csv')
    
    f1_cols = [i.lower().replace(" ", "_") for i in f1.columns]
    f1_cols[f1_cols.index("st")] = "state"
    
    f2_cols = [i.lower().replace(" ", "_") for i in f2.columns]
    f2_cols[f2_cols.index("st")] = "state"
    
    f3_cols = [i.lower().replace(" ", "_") for i in f3.columns]
    
    return f1_cols, f2_cols, f3_cols

cols_to_use = clean_columns()[0]

In [3]:
def load_original_data():
    """
    load original data with already sorted column order;
    concatenate data into one df
    """
    file1 = pd.read_csv(r'C:\Users\p.kollhof\Documents\IRONHACK_GitHub\IH_RH_DA_FT_AUG_2022\Class_Materials\Case_Studies\Customer_Analysis_Case_Study\Data\file1.csv')
    file2 = pd.read_csv(r'C:\Users\p.kollhof\Documents\IRONHACK_GitHub\IH_RH_DA_FT_AUG_2022\Class_Materials\Case_Studies\Customer_Analysis_Case_Study\Data\file2.csv')
    file3 = pd.read_csv(r'C:\Users\p.kollhof\Documents\IRONHACK_GitHub\IH_RH_DA_FT_AUG_2022\Class_Materials\Case_Studies\Customer_Analysis_Case_Study\Data\file3.csv')
    
    file1_cols, file2_cols, file3_cols = clean_columns()
    
    file1.columns = file1_cols
    file2.columns = file2_cols
    file3.columns = file3_cols
    
    file1 = file1[cols_to_use]
    file2 = file2[cols_to_use]
    file3 = file3[cols_to_use]
    
    return pd.concat([file1,file2,file3], axis=0) # axis = 0 means row-wise; 1 means col-wise

ca_df = load_original_data()

In [4]:
def clean_gender_col():
    """
    clean gender column data;
    make everything either M, F, or NaN
    """
    gender_list = []
    
    for gender in ca_df["gender"]:
        if type(gender)==float:
            gender_list.append(np.nan)
        elif type(gender)!=float:
            if gender.lower().startswith("f"):
                gender_list.append("F")
            else:
                gender_list.append("M")

    ca_df["gender"] = gender_list
    
clean_gender_col()

In [5]:
def clean_states_col():
    """
    clean state column data;
    
    """

    state_list = []

    for state in ca_df["state"]:
        if type(state)==str:
            if state.lower()=="az":
                state_list.append("Arizona")
            elif state.lower()=="wa":
                state_list.append("Washington")
            elif state.lower()=="cali":
                state_list.append("California")
            else:
                state_list.append(state)
        elif type(float):
            state_list.append(state)
    
    ca_df["state"] = state_list

clean_states_col()

In [6]:
def clean_education_col(to_replace, replace_with):
    """
    func descr
    """
    try:
        if to_replace in ca_df["education"]:
            ca_df["education"] = ca_df["education"].str.replace(to_replace, replace_with)
    except:
        pass


In [7]:
def drop_column(col):
    """
    take a column and delete it from the dataframe
    """
    try:
        if col not in ca_df.columns:
            raise ValueError(f"Your given column '{col}' is not a column in the dataframe!")
            drop_column()
    
        ca_df.drop(col, axis=1, inplace=True)
        
    except:
        pass


In [8]:
def clean_number_of_complaints():
    """
    func descr
    """
    # v1 - with list comprehension
    ca_df["number_of_open_complaints"] =  [int(entry.split("/")[1]) if type(entry)==str 
                                       else int(entry) if type(entry)!=float
                                       else entry
                                       for entry in ca_df["number_of_open_complaints"]]
    
    ## v2 - with lambda function
    #ca_df["number_of_open_complaints"] = list(map(lambda entry: int(entry.split("/")[1]) if type(entry)==str 
                                       #else int(entry) if type(entry)!=float
                                       #else entry , ca_df["number_of_open_complaints"]))

In [9]:
drop_column("customer")

ca_df.drop_duplicates(inplace=True)
ca_df.dropna(how="all", inplace=True)

clean_education_col("Bachelors", "Bachelor")
#ca_df["education"] = ["Bachelor" if x=="Bachelors" else x for x in ca_df["education"]]
#ca_df["education"] = ca_df["education"].replace("Bachelors", "Bachelor")

ca_df["customer_lifetime_value"] = [int(entry[:entry.rfind(".")-2]) if type(entry)!=float
                                    else int(entry) if entry >= 0
                                    else np.nan 
                                    for entry in ca_df["customer_lifetime_value"]]
# operation can also be performed by stripping "%" from string values, 
# then deviding by 100 via lambda function if type(entry)==str

ca_df.reset_index(drop=True, inplace=True)

In [10]:
ca_df

,state,gender,education,customer_lifetime_value,income,monthly_premium_auto,number_of_open_complaints,policy_type,vehicle_class,total_claim_amount
0,Washington,NaN,Master,NaN,0.00,1000.00,0,Personal Auto,Four-Door Car,2.70
1,Arizona,F,Bachelor,697953.59%,0.00,94.00,0,Personal Auto,Four-Door Car,1131.46
2,Nevada,F,Bachelor,1288743.17%,48767.00,108.00,0,Personal Auto,Two-Door Car,566.47
3,California,M,Bachelor,764586.18%,0.00,106.00,0,Corporate Auto,SUV,529.88
4,Washington,M,High School or Below,536307.65%,36357.00,68.00,0,Personal Auto,Four-Door Car,17.27
...,...,...,...,...,...,...,...,...,...,...
8870,California,M,Bachelor,23405.99,71941.00,73.00,0,Personal Auto,Four-Door Car,198.23
8871,California,F,College,3096.51,21604.00,79.00,0,Corporate Auto,Four-Door Car,379.20
8872,California,M,Bachelor,8163.89,0.00,85.00,3,Corporate Auto,Four-Door Car,790.78
8873,California,M,College,7524.44,21941.00,96.00,0,Personal Auto,Four-Door Car,691.20


hk_df['median_home_val'] =  pd.to_numeric(hk_df['median_home_val'], errors='coerce')

In [10]:
ca_df.isna().sum()

state                          0
gender                       122
education                      0
customer_lifetime_value        7
income                         0
monthly_premium_auto           0
number_of_open_complaints      0
policy_type                    0
vehicle_class                  0
total_claim_amount             0
dtype: int64

In [11]:
nulls_df = pd.DataFrame(round(ca_df.isna().sum()/len(ca_df),4)*100)
nulls_df = nulls_df.reset_index()
nulls_df.columns = ['header_name', '%_nulls']
nulls_df

,header_name,%_nulls
0,state,0.00
1,gender,1.37
2,education,0.00
3,customer_lifetime_value,0.08
4,income,0.00
5,monthly_premium_auto,0.00
6,number_of_open_complaints,0.00
7,policy_type,0.00
8,vehicle_class,0.00
9,total_claim_amount,0.00


In [12]:
ca_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8875 entries, 0 to 8874
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   state                      8875 non-null   object 
 1   gender                     8753 non-null   object 
 2   education                  8875 non-null   object 
 3   customer_lifetime_value    8868 non-null   float64
 4   income                     8875 non-null   float64
 5   monthly_premium_auto       8875 non-null   float64
 6   number_of_open_complaints  8875 non-null   int64  
 7   policy_type                8875 non-null   object 
 8   vehicle_class              8875 non-null   object 
 9   total_claim_amount         8875 non-null   float64
dtypes: float64(4), int64(1), object(5)
memory usage: 693.5+ KB


### Activity 2 (Tuesday)
- Replacing null values – Replace missing values with means of the column (for numerical columns). Pay attention that the Income feature for instance has 0s which is equivalent to null values. (We assume here that there is no such income with 0 as it refers to missing values)
Hint: numpy.nan is considered of float64 data type.
- Bucketing the data - Write a function to replace column "State" to different zones. California as West Region, Oregon as North West, and Washington as East, and Arizona and Nevada as Central
- (Optional) Standardizing the data – Use string functions to standardize the text data (lower case)

<b>Important: for Activity 3 and Activity 4 , please use the [file Data_Marketing_Customer_Analysis_Round3.csv](./Data/Data_Marketing_Customer_Analysis_Round3.csv) from the [Data](./Data) folder.</b>

In [13]:
ca_df["number_of_open_complaints"].astype("object")
cols_numeric = list(ca_df.select_dtypes(exclude=['object']).columns)
#cols_numeric.remove("number_of_open_complaints")
cols_numeric_nanmeans = [np.nanmean(ca_df[col]) for col in cols_numeric]
#cols_numeric_means_X = [sum([x for x in ca_df[col] if x>0])/len([x for x in ca_df[col] if x>0]) for col in cols_numeric]

def cols_replace_null_nan(cols_numeric):
    """
    replace 0 and NaN values in column with mean of that column;
    """
    for idx, col in enumerate(cols_numeric):
        ca_df[col].fillna(cols_numeric_nanmeans[idx], inplace=True)
        ca_df[col].replace(0, np.mean(ca_df[cols_numeric[idx]]), inplace=True)
        
cols_replace_null_nan(cols_numeric)

In [16]:
ca_df["gender"].unique()

array([nan, 'F', 'M'], dtype=object)

In [18]:
#df.select_dtypes(exclude=['object'])

In [19]:
def gender_replace_nan():
    """
    Replace unknown (nan) gender with "u";
    """
    #if "u" in ca_df["gender"]:
    ca_df["gender"] = list(map(lambda x: "u" if type(x)!=str else x , ca_df["gender"]))
    
gender_replace_nan()

In [20]:
def states_to_regions():
    """
    convert states to regions;
    """
    try:
        region_dict = {
            "California" : "West",
            "Oregon" : "North West",
            "Washington" : "East",
            "Arizona" : "Central",
            "Nevada" : "Central"
        }

        ca_df["state"] = [region_dict[state] if state in region_dict.keys()
                          else state
                          for state in ca_df["state"] ]
        ca_df.rename(columns={"state": "region"}, inplace=True)
    except:
        pass

states_to_regions()

In [21]:
def get_string_columns(df):
    """
    filter out all object (string) columns;
    return list of these columns;
    """
    object_strings_bool = [is_string_dtype(df[col]) for col in df.columns]
    object_strings_true = list(compress(list(ca_df.columns), object_strings_bool))
    
    return object_strings_true

In [22]:
def lowercase_col_text_data(entry):
    """
    Alternative version;
    apply lowercase to every entry in every column if type(entry)==string
    """
    if type(entry)==str:
        return entry.lower()
    else:
        return entry
    
ca_df = ca_df.applymap(lowercase_col_text_data)

In [23]:
#def lowercase_col_text_data():
    #"""
    #use get_string_columns() function to select string-containing columns;
    #cast every string-entry in such column as lowercase;
    #"""
    #string_columns = list(ca_df.select_dtypes(include=['object']).columns)

    #for col in string_columns:
        #ca_df[col] = [string.lower() if type(string)==str
                      #else string
                      #for string in ca_df[col]]
#    
#lowercase_col_text_data()

In [24]:
ca_df

,region,gender,education,customer_lifetime_value,income,monthly_premium_auto,number_of_open_complaints,policy_type,vehicle_class,total_claim_amount
0,east,u,master,7983.35,37823.32,1000.00,0,personal auto,four-door car,2.70
1,central,f,bachelor,6979.00,37823.32,94.00,0,personal auto,four-door car,1131.46
2,central,f,bachelor,12887.00,48767.00,108.00,0,personal auto,two-door car,566.47
3,west,m,bachelor,7645.00,37823.32,106.00,0,corporate auto,suv,529.88
4,east,m,high school or below,5363.00,36357.00,68.00,0,personal auto,four-door car,17.27
...,...,...,...,...,...,...,...,...,...,...
8870,west,m,bachelor,23405.00,71941.00,73.00,0,personal auto,four-door car,198.23
8871,west,f,college,3096.00,21604.00,79.00,0,corporate auto,four-door car,379.20
8872,west,m,bachelor,8163.00,37823.32,85.00,3,corporate auto,four-door car,790.78
8873,west,m,college,7524.00,21941.00,96.00,0,personal auto,four-door car,691.20


<b>Important: for Activity 3 and Activity 4 , please use the [file Data_Marketing_Customer_Analysis_Round3.csv](./Data/Data_Marketing_Customer_Analysis_Round3.csv) from the [Data](./Data) folder.</b>

### Activity 3 (Wednesday)

- Get the numeric data into dataframe called `numerical` and categorical columns in a dataframe called `categoricals`.
(You can use np.number and np.object to select the numerical data types and categorical data types respectively)
- Now we will try to check the normality of the numerical variables visually
  - Use seaborn library to construct distribution plots for the numerical variables
  - Use Matplotlib to construct histograms
  - Do the distributions for different numerical variables look like a normal distribution 
- For the numerical variables, check the multicollinearity between the features. Please note that we will use the column `total_claim_amount` later as the target variable.
- Optional: Drop one of the two features that show a high correlation between them (greater than 0.9). If there is no pair of features that have a high correlation, then do not drop any features.

### Activity 4 (Thursday)

- Show a plot of the total number of responses.
- Show a plot of the response by the sales channel.
- Show a plot of the response by the total claim amount.
- Show a plot of the response by income.
- (Optional) Don't limit your creativity!  plot any interesting findings/insights that describe some interesting facts about your data set and its variables.
- Plot the Correlation Heatmap.
- Clean your notebook and make it a readible and presentable with a good documentation that summarizes the Data Cleaning, Exploration(including plots) Steps that you have performed.